In [1]:
import cv2
import os
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'keras'

In [ ]:
IMAGE_WIDTH=200
IMAGE_HEIGHT=200
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
BATCH_SIZE=32
EPOCHS=50

In [ ]:
filenames_with = os.listdir("../input/dataset-for-mask-detection/dataset/with_mask")
filenames_without = os.listdir("../input/dataset-for-mask-detection/dataset/without_mask")
filenames_list_with = []
filenames_list_without = []
categories_with = []
categories_without = []
for filename in filenames_with:
    filenames_list_with.append("../input/dataset-for-mask-detection/dataset/with_mask/" + filename)
    categories_with.append(str(1))
for filename in filenames_without:
    filenames_list_without.append("../input/dataset-for-mask-detection/dataset/without_mask/" + filename)
    categories_without.append(str(0))
    

df_w = pd.DataFrame({
    'image': filenames_list_with,
    'category': categories_with
})
df_wo = pd.DataFrame({
    'image': filenames_list_without,
    'category': categories_without
})
print(df_w.shape, df_wo.shape)
df = df_w.append(df_wo)
print(df)

In [ ]:
#split data into train and valid set
train_df, valid_df = train_test_split(df, test_size = 0.15, stratify = df['category'], random_state = 3)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
total_train = train_df.shape[0]
total_valid = valid_df.shape[0]
print(train_df.shape)
print(valid_df.shape)

In [ ]:
#We'll perform individually on train and validation set.
train_datagen = ImageDataGenerator(
#                                    zoom_range = 0.2,
                                   rescale=1./255,
                                   horizontal_flip = True,
                                   )
#fill_mode : 이미지를 회전, 이동하거나 축소할 때 생기는 공간을 채우는 방식

train_gen = train_datagen.flow_from_dataframe(train_df,
                                              x_col = 'image',
                                              y_col = 'category',
                                              target_size = IMAGE_SIZE,
                                              batch_size = BATCH_SIZE,
                                              class_mode='binary',
                                              validate_filenames=False
                                             )

#we do not augment validation data.
validation_datagen = ImageDataGenerator(rescale=1./255)
valid_gen = validation_datagen.flow_from_dataframe(
    valid_df, 
    x_col="image",
    y_col="category",
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=BATCH_SIZE,
    validate_filenames=False
)


In [ ]:
def create_model():
    model = Sequential()
    model.add(ResNet50(include_top = False, pooling = 'max', weights = 'imagenet'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.layers[0].trainable = False 
    model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'binary_crossentropy')    
    return model


In [ ]:
model = create_model()

In [ ]:
model.summary()

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
checkpointer = ModelCheckpoint(filepath = 'mask.weights.best.hdf5', save_best_only = True, save_weights_only = True)
callbacks = [learning_rate_reduction]

In [ ]:
model.fit_generator(train_gen,
                    epochs = EPOCHS,
                    validation_data = valid_gen,
                    validation_steps=total_valid//BATCH_SIZE,
                    steps_per_epoch=total_train//BATCH_SIZE,
                    callbacks = callbacks)

In [ ]:
loss = pd.DataFrame(model.history.history)
loss[['loss', 'val_loss']].plot()
loss[['accuracy', 'val_accuracy']].plot()